<a href="https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/marta/agentic_shopping_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Agentic Shopping Assistant

This notebook will go through all the steps to create an agentic shopping assistant. \
We will:
1. Connect to OpenAI
2. Create a simple agent
3. Create an MCP server
4. Create a LangGraph agent

<br/>
<img src="https://github.com/marta-manzin/agentic-shopping-assistant/blob/marta/images/assistant.png?raw=1" width="500">


# ⚙️ Setup

Setup
Before we start using OpenAI models, you need to set an API key. \
If you don't already have an key, you can generate one at: https://platform.openai.com/api-keys. \

Save the key as a Colab Secret variable called "OPENAI_API_KEY":
1. Click on the key icon in the left bar menu.
2. Click on `+ Add new secret`.
3. Name the variable and paste the key in the value field.
4. Enable notebook access.

<br/>
<img src="https://github.com/marta-manzin/agentic-shopping-assistant/blob/marta/images/colab_setup.png?raw=1" width="450">




Import the API key into the notebook.

In [1]:
import os

# Detect if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set API key based on environment
if IN_COLAB:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
else:
    # For local Jupyter: ensure OPENAI_API_KEY is set in your environment
    if "OPENAI_API_KEY" not in os.environ:
        print("Warning: OPENAI_API_KEY not found in environment variables")

Then, make a test call to OpenAI.

In [4]:
import openai
client = openai.OpenAI()
model = "gpt-4o"

# Test that the LLM is set up correctly
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say 'OK' if you can read this."}],
    max_tokens=10
)
print(f"LLM test: {response.choices[0].message.content}")

LLM test: OK


# 🤖 Creating an Agent

In Python, a set is an unordered collection of unique elements. \
We will build an agent that adds and removes strings from a set.

## Defining the tools

The System Prompt gives context to the LLM

In [5]:
SYSTEM_PROMPT = """
You are a helpful assistant that adds and removes strings from a set.

You have access to tools that let you:
1. Add a string, if it is not already in the set.
2. Remove a string.
3. Read all contents of the set.
"""

Here are the available tools:

In [6]:
MY_SET = set()

def insertion_tool(s: str) -> str:
  """Tool: Add a string to a set."""
  try:
    MY_SET.add(s)
    return f"Inserted '{s}'."
  except Exception as ex:
    return f"Failed to insert '{s}'. {ex!r}"

def removal_tool(s: str) -> str:
  """Tool: Remove a string from a set."""
  try:
    if s in MY_SET:
      MY_SET.remove(s)
      return f"Removed '{s}'."
    else:
      return f"'{s}' is not in the set."
  except Exception as ex:
    return f"Failed to remove '{s}'. {ex!r}"

def get_set_tool() -> str:
  """Tool: Get the contents of the set."""
  try:
    if MY_SET:
      return f"The set contains: {sorted(MY_SET)}"
    else:
      return "The set is empty."
  except Exception as ex:
    return f"Failed to get set contents. {ex!r}"

Provide a description of each tool to the LLM.

In [7]:
tools: list[dict] = [
    {
        "type": "function",
        "function": {
            "name": "insertion_tool",
            "description": "Add a string to a set.",
            "parameters": {
                "type": "object",
                "properties": {
                    "s": {
                        "type": "string",
                        "description": "The string to be added."
                    },
                },
                "required": ["s"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "removal_tool",
            "description": "Remove a string from a set.",
            "parameters": {
                "type": "object",
                "properties": {
                    "s": {
                        "type": "string",
                        "description": "The string to be removed."
                    },
                },
                "required": ["s"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_set_tool",
            "description": "Get the contents of the set.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
]

## Calling a Tool

If the LLM decides to run a tool, instead of responding with a message, it will respond with a `tool_call` object.

In [8]:
from types import SimpleNamespace

# Create a tool_call object that matches OpenAI's structure
tool_call = SimpleNamespace(
    id="call_abc123",
    function=SimpleNamespace(
        name="insertion_tool",
        arguments='{"s":"apple"}'
    ),
    type="function"
)

Extract the function name from the tool call.

In [9]:
function_name = tool_call.function.name
function_name

'insertion_tool'

Parse the arguments from JSON string to dictionary.

In [10]:
import json

arguments = json.loads(tool_call.function.arguments)
arguments

{'s': 'apple'}

Important! Verify that the function is one of the allowed tools.

In [11]:
allowed_tool_names = [tool["function"]["name"] for tool in tools]
if function_name not in allowed_tool_names:
    print(f"Error: '{function_name}' is not an allowed tool.")
else:
    print(f"'{function_name}' is an allowed tool.")


'insertion_tool' is an allowed tool.


Verify that the function exists and is callable.

In [12]:
tool_func = globals().get(function_name)
if tool_func is None or not callable(tool_func):
    print(f"Unknown function: {function_name}")
else:
    print(f"Function {function_name} exists.")

Function insertion_tool exists.


Call the function with the unpacked arguments and print its response.

In [13]:
MY_SET = set()

response = tool_func(**arguments)
response

"Inserted 'apple'."

Combine all tool calling steps in one method.

In [14]:
import json

def execute(tool_call) -> str:
    """Execute a tool call and return the result, if any."""
    # Extract the function name from the tool call
    function_name = tool_call.function.name

    # Parse the arguments from JSON string to dictionary
    arguments = json.loads(tool_call.function.arguments)

    # Important! Verify that the function is one of the allowed tools
    allowed_tool_names = [tool["function"]["name"] for tool in tools]
    if function_name not in allowed_tool_names:
        return f"Error: '{function_name}' is not an allowed tool."

    # Verify that the function exists and is callable
    tool_func = globals().get(function_name)
    if tool_func is None or not callable(tool_func):
        return f"Unknown function: {function_name}"

    # Call the function with the unpacked arguments
    response = tool_func(**arguments)

    # Return the tool's response
    return response

## The agentic loop
Instead of using a ready-made framework, the code below implements *direct orchestration*.

We want to build a `while True` loop with tool calls when the LLM requests it.  

<img src="https://github.com/marta-manzin/agentic-shopping-assistant/blob/marta/images/agentic_flow.png?raw=1" width="600">

`print_history` is a utility function that will display the LLM conversation history.

In [15]:
import textwrap

def print_history(messages, width=70):
    """Pretty print messages history."""
    # For each message in the history
    for i, msg in enumerate(messages):
        print(f"\n[{i}] Role: {msg['role']}")

        # Display message text if present
        if msg.get('content'):
            for line in str(msg['content']).split('\n'):
                wrapped = textwrap.fill(line, width=width, initial_indent='    ', subsequent_indent='    ')
                print(wrapped)

        # Display tool calls if present
        if msg.get('tool_calls'):
            for tc in msg['tool_calls']:
                func_name = tc.function.name
                func_args = tc.function.arguments
                print(f"    🔧 {func_name}({func_args})")

        # If this is an assistant message with no tool calls, show completion
        if msg['role'] == 'assistant' and not msg.get('tool_calls'):
            print(f"\n⭐ The resulting set is: {MY_SET}")

Initialize the chat history.

In [16]:
messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Please add 'apples' to the set."
    }
]

print_history(messages)


[0] Role: system

    You are a helpful assistant that adds and removes strings from a
    set.

    You have access to tools that let you:
    1. Add a string, if it is not already in the set.
    2. Remove a string.
    3. Read all contents of the set.


[1] Role: user
    Please add 'apples' to the set.


Ask the agent what to do next.

In [17]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
).choices[0].message

# Display the raw tool call in the agent's response
tool_calls_data = [tc.model_dump() for tc in response.tool_calls]
print(json.dumps(tool_calls_data, indent=2))

[
  {
    "id": "call_ezh1aLZZupCly1ID5tA7VgdX",
    "function": {
      "arguments": "{\"s\":\"apples\"}",
      "name": "insertion_tool"
    },
    "type": "function"
  }
]


Update the chat history with the agent's response.

In [18]:
messages.append({
    "role": "assistant",
    "content": response.content,
    "tool_calls": response.tool_calls
})

print_history(messages)


[0] Role: system

    You are a helpful assistant that adds and removes strings from a
    set.

    You have access to tools that let you:
    1. Add a string, if it is not already in the set.
    2. Remove a string.
    3. Read all contents of the set.


[1] Role: user
    Please add 'apples' to the set.

[2] Role: assistant
    🔧 insertion_tool({"s":"apples"})


Execute the tool call.

In [19]:
MY_SET = set()

outcome = execute(response.tool_calls[0])
outcome

"Inserted 'apples'."

Append the outcome to the message history.

In [20]:
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(outcome)
})
print_history(messages)


[0] Role: system

    You are a helpful assistant that adds and removes strings from a
    set.

    You have access to tools that let you:
    1. Add a string, if it is not already in the set.
    2. Remove a string.
    3. Read all contents of the set.


[1] Role: user
    Please add 'apples' to the set.

[2] Role: assistant
    🔧 insertion_tool({"s":"apples"})

[3] Role: tool
    Inserted 'apples'.


Combine all direct orchestration steps into one method.

In [21]:
def submit_request(
    user_prompt: str,
    verbose: bool = True
    ):
    """Submit a request to the agent and run any tools it calls."""
    # Initialize the chat history
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    while True:

        # Ask the agent what to do next
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        ).choices[0].message

        # Update the chat history with the agent's response
        messages.append({
            "role": "assistant",
            "content": response.content,
            "tool_calls": response.tool_calls
        })

        # If agent did not call any tools, we are done
        if not response.tool_calls:
            if verbose:
              print(f"\n⭐ The resulting set is: {MY_SET}")
            break

        # Execute all tool calls
        for tool_call in response.tool_calls:
            if verbose:
              print(f"\n🔧 The agent is calling a tool: "
                  f"{tool_call.function.name}"
                  f"({json.loads(tool_call.function.arguments)})")

            # Append the outcome to the message history
            outcome = execute(tool_call)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(outcome)
            })

## Let's test the agent!

Submit a request to the agent.

In [22]:
MY_SET = set()
submit_request("Please add 'apples', 'oranges' and 'pears' to the set.")


🔧 The agent is calling a tool: insertion_tool({'s': 'apples'})

🔧 The agent is calling a tool: insertion_tool({'s': 'oranges'})

🔧 The agent is calling a tool: insertion_tool({'s': 'pears'})

⭐ The resulting set is: {'oranges', 'apples', 'pears'}


Inspect the message history.

In [23]:
submit_request("Please remove 'oranges' from the set.")


🔧 The agent is calling a tool: removal_tool({'s': 'oranges'})

⭐ The resulting set is: {'apples', 'pears'}


# 🗄️ Creating an MCP Server

Create the MCP server.

In [24]:
%pip install --quiet mcp
from mcp.server import Server
from mcp.types import Tool, TextContent

server = Server("set-server")
print("✓ Server created")

✓ Server created


Create an MCP wrapper for listing the available tools.

In [25]:
async def list_tools() -> list[Tool]:
    """Return the list of available tools from our tools definition."""
    # Create an empty list to store MCP Tool objects
    mcp_tools = []

    # Convert each tool from our OpenAI format to MCP format
    for tool_def in tools:
        # Extract the function definition from the OpenAI tool format
        func_def = tool_def["function"]

        # Create an MCP Tool object with the same information
        mcp_tools.append(Tool(
            name=func_def["name"], # the function name
            description=func_def["description"], # what the tool does
            inputSchema=func_def["parameters"] # the JSON schema for parameters
        ))

    # Return the list of MCP Tool objects
    return mcp_tools

# Register the list_tools function with the server
server.list_tools()(list_tools)

<function __main__.list_tools() -> list[mcp.types.Tool]>

Create an MCP wrapper for executing tools.

In [26]:
from types import SimpleNamespace

async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    """Handle MCP tool calls by delegating to our existing tools."""
    # Convert MCP format to the format expected by execute()
    function = SimpleNamespace(
        name=name,
        arguments=json.dumps(arguments)  # Convert dict to JSON string
    )
    tool_call = SimpleNamespace(function=function)

    # Execute the tool using the existing execute() function
    result = execute(tool_call)

    # Convert result to MCP response format
    return [TextContent(type="text", text=str(result))]

# Register the call_tool function with the server
server.call_tool()(call_tool)

<function __main__.call_tool(name: str, arguments: dict) -> list[mcp.types.TextContent]>

Create a web application.

In [27]:
# FastAPI is a framework for building REST APIs
%pip install --quiet fastapi
from mcp.server.sse import SseServerTransport
from fastapi import FastAPI, Request
from fastapi.responses import Response

# Create a FastAPI web application
app = FastAPI()

✓ FastAPI app created


Expose an HTTP/SSE POST endpoint, used to establish the connection to the server.

In [ ]:
# Create an SSE transport that will handle messages at the "/messages" path
sse = SseServerTransport("/messages")

# Mount the POST handler for receiving messages
# Clients send messages to http://host:port/messages
app.mount("/messages", sse.handle_post_message)

Expose a GET endpoint, used to handle incoming tool calls.

In [ ]:
async def handle_sse(request: Request):
    """Handle incoming SSE connections from MCP clients."""
    # Connect the SSE transport to get read/write streams
    async with sse.connect_sse(
        request.scope, request.receive, request._send
    ) as (read_stream, write_stream):
        # Run the MCP server with these streams
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )
    return Response()

# Register the GET endpoint with the FastAPI app
# Clients connect to http://host:port/sse to establish SSE connection
app.add_api_route("/sse", handle_sse, methods=["GET"])

Start the MCP server in the background.

In [28]:
# Uvicorn is a web server that handles HTTP requests and asynchronous code
%pip install --quiet uvicorn
import threading
import uvicorn
import sys

# The port number where the server will listen
server_port = 12345

def run_server():
    """Run the uvicorn server. This will be called in a background thread."""
    try:
        # Start the server on all network interfaces (0.0.0.0) at the specified port
        uvicorn.run(app, host="0.0.0.0", port=server_port, log_level="warning")
    except Exception as e:
        # Print any errors to stderr
        print(f"✗ Server error: {e}", file=sys.stderr)

# Start server in background thread
print(f"Starting MCP HTTP server on port {server_port} in background...")
server_thread = threading.Thread(
    target=run_server,
    daemon=True  # thread will automatically stop when main program exits
)
server_thread.start()

# Confirm server URL
server_url = f"http://127.0.0.1:{server_port}/sse"
print(f"Server available at {server_url}")

Starting MCP HTTP server on port 12345 in background...
  Server available at http://127.0.0.1:12345/sse


# 🤝 Creating an MCP Client

List available tools on the MCP server.

In [30]:
from mcp import ClientSession
from mcp.client.sse import sse_client

# Utility function to list tools
async def list_mcp_tools():
    """Helper function to list available MCP tools."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            available_tools = await session.list_tools()
            return available_tools

available_tools = await list_mcp_tools()
print("Available tools:", [t.name for t in available_tools.tools])

Available tools: ['insertion_tool', 'removal_tool', 'get_set_tool']


Test each tool.

In [35]:
# Utility function to call a tool
async def call_mcp_tool(tool_name: str, arguments: dict = {}):
    """Helper function to call an MCP tool."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, arguments)
            return result

MY_SET = set()

In [36]:
# Test the insertion_tool by adding 'cherries' to the set
result = await call_mcp_tool("insertion_tool", {"s": "cherries"})
print(result.content[0].text)

Inserted 'cherries'.


In [37]:
# Test the removal_tool by removing 'Bananas' from the set
result = await call_mcp_tool("removal_tool", {"s": "bananas"})
print(result.content[0].text)

'bananas' is not in the set.


In [38]:
# Test the get_set_tool
result = await call_mcp_tool("get_set_tool")
print(result.content[0].text)

The set contains: ['cherries']


# 🧠 Orchestration with LangGraph

In [ ]:
%pip uninstall -y -qqq langchain
%pip install --quiet "langchain-openai>=0.2,<1.0" "langchain_mcp_adapters" "langgraph"

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Create MCP client that connects to your set-server
client = MultiServerMCPClient(
    {
        "set-server": {
            "transport": "sse",
            "url": f"http://localhost:{server_port}/sse",
        }
    }
)

In [ ]:
# Get available tools from the MCP server
tools_from_mcp = await client.get_tools()
print(f"✓ Loaded {len(tools_from_mcp)} tools from MCP server")
for tool in tools_from_mcp:
    print(f"  - {tool.name}: {tool.description}")

In [ ]:
if IN_COLAB:
    tools_from_mcp = asyncio.run(client.get_tools())

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# Create a LangGraph agent using the tools we already loaded
agent_executor = create_react_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    tools_from_mcp,
)

print("✓ LangGraph agent created")

In [ ]:
result = await agent_executor.ainvoke({
    "messages": [{"role": "user", "content": "Please add 'grapes', 'kiwi', and 'mango' to the set."}]
})

In [ ]:
if IN_COLAB:
    result = asyncio.run(
        agent_executor.ainvoke({
            "messages": [{
                "role": "user",
                "content": "Please add 'grapes', 'kiwi', and 'mango' to the set."}]
        })
    )

In [ ]:
# Display the conversation
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

for message in result['messages']:
    if isinstance(message, HumanMessage):
        print("Human: \033[32m" + message.content + "\033[0m")
    elif isinstance(message, AIMessage):
        if message.content:
            print("AI: \033[34m" + message.content + "\033[0m")
    elif isinstance(message, ToolMessage):
        if "Error" not in message.content:
            print(f"Tool Result: \033[32mSuccess\033[0m")

print(f"\n⭐ The resulting set is: {MY_SET}")

# 🧹 Cleanup

Stop the MCP server.

In [ ]:
# Kill any process running uvicorn on our server port
!pkill -f "uvicorn.*{server_port}"
print("✓ Server stopped")

# Thank you!

###